<a href="https://colab.research.google.com/github/zhlu97/pn_pc_cls/blob/master/EasyPointNet_Default_Model8000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Easiest PointNet Possible

## Fetch ModelNet40 and Decompress

In [ ]:
!wget https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip

--2020-04-21 18:22:50--  https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip
Resolving shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)... 171.67.77.19
Connecting to shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)|171.67.77.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1705117335 (1.6G) [application/zip]
Saving to: ‘modelnet40_normal_resampled.zip’

modelnet40_normal_r 100%[===================>]   1.59G  3.34MB/s    in 14m 21s 

2020-04-21 18:37:12 (1.89 MB/s) - ‘modelnet40_normal_resampled.zip’ saved [1705117335/1705117335]



In [ ]:
%%capture
!unzip modelnet40_normal_resampled.zip

## Import Libraries

In [ ]:
import numpy as np
import warnings
import os
from torch.utils.data import Dataset
warnings.filterwarnings('ignore')

import torch
import logging
# from tqdm import tqdm
from tqdm.notebook import tqdm
import sys
import importlib

import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable

import torch.nn.functional as F

## Define hyperparameters

In [ ]:
batch_size = 60
sample_point_count = 1024
num_classes = 40
num_epoch = 66
learning_rate = 0.001
# rootPath = '/content/modelnet40_normal_resampled/'
rootPath = '/home/haokun/Downloads/modelnet40_normal_resampled'

## Dataloader

### To use Pytorch's built-in dataloader, must implement subclass and override all three functions below.

In [ ]:
class ModelNetResampledLoader(Dataset):
  def __init__(self, rootPath, mode):
        self.categoryNames = [line.rstrip() for line in open(os.path.join(rootPath, 'modelnet40_shape_names.txt'))]
        self.classes = dict(zip(self.categoryNames, range(len(self.categoryNames))))

        self.all_IDs = {}
        self.all_IDs['train'] = [line.rstrip() for line in open(os.path.join(rootPath, 'modelnet40_train.txt'))]
        self.all_IDs['test']  = [line.rstrip() for line in open(os.path.join(rootPath, 'modelnet40_test.txt'))]   

        shape_names = ['_'.join(x.split('_')[0:-1]) for x in self.all_IDs[mode]]
        self.datapath = [(shape_names[i], os.path.join(rootPath, shape_names[i], self.all_IDs[mode][i]) + '.txt') for i
                         in range(len(self.all_IDs[mode]))]        

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.datapath)

  def __getitem__(self, index):
        selectedDatapaths = self.datapath[index]
        selectedClasses   = self.classes[selectedDatapaths[0]]
        selectedClasses   = np.array([selectedClasses]).astype(np.int32)

        pointSet = np.loadtxt(selectedDatapaths[1], delimiter=',').astype(np.float32)
        pointSet = pointSet[8000:8000+sample_point_count,:] #Only choosing first 1024? -- Points have no ordering. We can already see shape with first 1024 pts
        pointSet = pointSet[:, 0:3]

        return pointSet, selectedClasses

## Simpliest PointNet Model

In [ ]:
class STNkd(torch.nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = torch.nn.Linear(1024, 512)
        self.fc2 = torch.nn.Linear(512, 256)
        self.fc3 = torch.nn.Linear(256, k * k)
        self.relu = torch.nn.ReLU()

        self.bn1 = torch.nn.BatchNorm1d(64)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.bn3 = torch.nn.BatchNorm1d(1024)
        self.bn4 = torch.nn.BatchNorm1d(512)
        self.bn5 = torch.nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1, self.k * self.k).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class TNet(torch.nn.Module):
    def __init__(self, channel=3):
        super(TNet, self).__init__()
        self.channel = channel
        self.affine = torch.nn.Linear(channel, channel)

    def forward(self, x):
        print(torch.transpose(x, 2, 1))
        return self.affine(torch.transpose(x, 2, 1))

class PointNetEncoder(torch.nn.Module):
    def __init__(self, channel=3):
        super(PointNetEncoder, self).__init__()

        # self.tNet3 = TNet(channel)
        self.tNet3 = STNkd(channel)
        self.featureExpansion_3_64 = torch.nn.Sequential(
            torch.nn.Conv1d(channel, 64, 1),
            torch.nn.BatchNorm1d(64)
        )
        # self.tNet64 = TNet(64)
        self.tNet64 = STNkd(64)
        self.featureExpansion_64_128_1024 = torch.nn.Sequential(
            torch.nn.Conv1d(64, 64, 1),
            torch.nn.BatchNorm1d(64),
            torch.nn.Conv1d(64, 128, 1),
            torch.nn.BatchNorm1d(128),
            torch.nn.Conv1d(128, 1024, 1),
            torch.nn.BatchNorm1d(1024)
        )

    def forward(self, x):
        B, D, N = x.size()

        inputTransform = self.tNet3(x)
        
        x = x.transpose(2, 1)
        x = torch.bmm(x, inputTransform)
        x = x.transpose(2, 1)

        x = self.featureExpansion_3_64(x)
        featTranform = self.tNet64(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, featTranform)
        x = x.transpose(2, 1)

        pointfeat = x
        x = self.featureExpansion_64_128_1024(x)
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        return x, inputTransform, featTranform

class Model(torch.nn.Module):
    def __init__(self, channel = 3):
        super(Model, self).__init__()

        self.pointNetEncoder = PointNetEncoder(channel=channel)

        self.pointNetDecoder = torch.nn.Sequential(
            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.Dropout(p=0.4),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, num_classes),
            torch.nn.Softmax(dim = 1)
        )

    def forward(self, x):
        x, inputTransform, featTranform = self.pointNetEncoder(x)
        x = self.pointNetDecoder(x)

        return x

class Loss(torch.nn.Module):
    def __init__(self):
        super(Loss, self).__init__()

    def forward(self, pred, target):
        loss = torch.nn.functional.nll_loss(pred, target)
        return loss

## Training

In [ ]:
# Get data loader
trainData = ModelNetResampledLoader(rootPath=rootPath, mode='train')
testData  = ModelNetResampledLoader(rootPath=rootPath, mode='test')
trainDataLoader = torch.utils.data.DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=4)
testDataLoader  = torch.utils.data.DataLoader(testData, batch_size=batch_size, shuffle=False, num_workers=4)

# Get model
model = Model().cuda()
criterion = Loss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Print model's state_dict
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# # Print optimizer's state_dict
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
print('Start training... Batch size： %d' % batch_size)
epoch_accuracy = []
epoch_loss = []
for epoch in range(0,num_epoch):
    correctRates = []
    lossCur = []
    for batch_id, data in tqdm(enumerate(trainDataLoader, 0), total=len(trainDataLoader)):
        points, target = data
        points = points.data.numpy()
        points = torch.Tensor(points)
        target = target[:, 0]

        points = points.transpose(2, 1)
        points, target = points.cuda(), target.cuda()
        optimizer.zero_grad()

        model = model.train()
        pred = model(points)
        loss = criterion(pred, target.long())
        lossCur.append(loss.data.cpu().numpy())
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.long().data).cpu().sum()
        correctRates.append(correct.item() / float(points.size()[0]))
        loss.backward()
        optimizer.step()

    trainEpochAccuracy = np.mean(correctRates)
    print('Train Epoch %d Accuracy: %f' %(epoch, trainEpochAccuracy))
    epoch_accuracy.append(trainEpochAccuracy)
    epoch_loss.append(np.exp(np.mean(lossCur))) # what we are using is negative log likelihood loss

torch.save(model, 'model.pytorch')

np.save("epoch_accuracy.npy", epoch_accuracy)
np.save("epoch_loss.npy", epoch_loss)

## Testing

In [ ]:
# Model class must be defined somewhere
model = torch.load('model.pytorch')
model.eval()

mean_correct = []
class_acc = np.zeros((num_classes,3))
for batch_id, data in tqdm(enumerate(testDataLoader), total=len(testDataLoader)):
    points, target = data
    target = target[:, 0]
    points = points.transpose(2, 1)
    points, target = points.cuda(), target.cuda()
    classifier = model.eval()
    pred = classifier(points)
    pred_choice = pred.data.max(1)[1].type(torch.cuda.IntTensor)
    equalCount = (pred_choice == target).cpu().sum()
    mean_correct.append(equalCount.item() / float(points.size()[0]))
instance_acc = np.mean(mean_correct)
print('Test Instance Accuracy: %f' % instance_acc)


Test Instance Accuracy: 0.497222
